https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7924484/

https://codingfleet.com/transformation-details/determining-gender-by-name-with-python/

In [1]:
%%writefile gender_guesser_first_names.py
import gender_guesser.detector as GENDER

class Detector:
    '''
    * Works for all countries
    * convert mostly_(fe)male to (fe)male
    * Works for composed names by finding the first female or male and ignoring the other ones
    '''
    LA_countries=['Brazil', 'Mexico', 'Argentina','Chile', 'Colombia','Bolivia','Cuba', #5
              'Costa Rica', 'Ecuador', 'El Salvador', 'Guatemala', 'Honduras', #10
               'Nicaragua', 'Panama', 'Paraguay', 'Peru', #15
              'Dominican Republic','Uruguay','Venezuela']
    
    def __init__(self,case_sensitive=False):
        self.case_sensitive = case_sensitive
    #case_sensitive = False
        self.D=GENDER.Detector(case_sensitive=self.case_sensitive)

    def get_country(self,country):
        if country.lower() in [s.lower() for s  in self.LA_countries if s.lower()!='brazil']:
            return 'spain'
        elif country.lower() == 'brazil':
            return 'portugal'
        elif country.lower() in self.D.__class__.COUNTRIES:
            return country.lower()
        else:
            return None

    def get_gender_country(self,name,country=None):
        if country:
            country = self.get_country(country)
            fm=self.D.get_gender(name,country)
            fm=fm.replace('mostly_','')
        else:
            fm='unknown'
        if fm not in ['male','female']:
            fm = self.D.get_gender(name)
            fm=fm.replace('mostly_','')
        return fm  
    
    def get_gender(self,names,country=None):
        for name in names.split():
            fm = self.get_gender_country(name,country=country)
            if fm in ['male','female']:
                break
        return fm

            
d=Detector()
assert d.get_gender('Andrea',country='italy') == 'male'
assert d.get_gender('Andrea',country='spain') == 'female'
assert d.get_gender('Andrea',country='colombia') == 'female'
assert d.get_gender('Diego',country='colombia') == 'male'
assert d.get_gender('Alejandro',country='colombia') == 'male'
assert d.get_gender('Diego Alejandro',country='colombia') == 'male'
# https://www.reddit.com/r/namenerds/comments/178mdjr/names_that_are_different_genders_in_different/
assert d.get_gender('Valery',country='russia') == 'male'
assert d.get_gender('Valery',country='usa') == 'female'
assert d.get_gender('Marian',country='poland') == 'male'
assert d.get_gender('Marian',country='usa') == 'female'
assert d.get_gender('Kim',country='denmark') == 'male'
assert d.get_gender('Kim',country='spanish') == 'female'
assert d.get_gender('Jan',country='germany') == 'male'
assert d.get_gender('Jan',country='usa') == 'female'

Overwriting gender_guesser_first_names.py


In [2]:
import gender_guesser_first_names as gender 

In [3]:
d=gender.Detector(case_sensitive=False)

In [4]:
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/colav-playground/scienti/main/data/test_gender.csv')
#Add special cases
df = pd.concat([df,   pd.DataFrame([{'names':'Andrea','country':'italy','gender':'male'}])])

df['gg'] = df.apply(lambda x: d.get_gender( x.get('names'),
                  country= x.get('country') ) ,axis='columns')

assert df.shape[0] == 3968

T=df[df['gender']==df['gg']].shape[0]
assert T == 3630

U=df[df['gg']=='unknown'].shape[0]
assert U == 298

E=df[ (df['gender']!=df['gg']) & (df['gg']!='unknown') ][['names',
                                                          'country','gender','gg']].shape[0]
assert E == 40

In [5]:
E/T #0.13 to 0.1 with Spain

0.011019283746556474

In [6]:
U/T 

0.08209366391184574